Various GWAS genesets are tested for enrichment in SCZ somatic variants.

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from bsmcalls import SNPnexus
from bsmcalls import operations
from matplotlib import pyplot as plt
import attila_utils
%matplotlib inline

In [2]:
data = SNPnexus.load_data('/home/attila/projects/bsm/results/2020-09-07-annotations/annotated-calls.p')

Read autism GWAS loci from the GWAS Catalog to a DataFrame

In [3]:
gwaspath = '/home/attila/projects/bsm/resources/GWAS/autism-spectrum-disorder.tsv'
asdloci = pd.read_csv(gwaspath, sep='\t')

We will select the study ``Identification of common genetic risk variants for autism spectrum disorder.'' (PMID [30804558](www.ncbi.nlm.nih.gov/pubmed/30804558)).

In [4]:
asdloci = asdloci.loc[asdloci['PUBMEDID'] == 30804558]
selcols = ['CHR_ID', 'CHR_POS', 'REPORTED GENE(S)', 'MAPPED_GENE']
asdloci[selcols]

,CHR_ID,CHR_POS,REPORTED GENE(S),MAPPED_GENE
94,1,96096246,"LOC102723661, PTBP2",LINC02790 - RNU1-130P
95,1,96513405,Intergenic,EEF1A1P11 - RN7SL831P
96,1,98627228,Intergenic,LINC01776 - AC095031.1
97,1,104249635,Intergenic,AC092506.1 - AL591888.1
98,1,193533479,Intergenic,AL136456.1
...,...,...,...,...
151,20,21267478,"KIZ, XRN2, NKX2-2, NKX2-4",ZNF877P - AL117332.1
152,5,104676602,NaN,AC099520.1
153,14,103551616,NaN,RNU7-160P - AL139300.2
154,1,72263459,NaN,NEGR1


In [5]:
reported = set(asdloci['REPORTED GENE(S)'].dropna().str.split(', ').sum())
reported.remove('Intergenic')
reported

{'C8orf74',
 'CADPS',
 'KIZ',
 'KMT2E',
 'LOC102723661',
 'MACROD2',
 'MROH5',
 'NKX2-2',
 'NKX2-4',
 'PINX1',
 'PTBP2',
 'SOX7',
 'SRPK2',
 'XRN2'}

In [6]:
mappeds = asdloci['MAPPED_GENE'].dropna()
commas = mappeds.str.match('^.*, .*$')
mappeds1 = mappeds.loc[commas]
mapped1 = set(mappeds1.str.split(', ').sum())
mappeds2 = mappeds.loc[~ commas]
mapped2 = set(mappeds2.str.split(' - ').sum())
mapped = mapped2.union(mapped1)
asd_gwas_genes = mapped.union(reported)
print(len(asd_gwas_genes), 'genes')
print(asd_gwas_genes)

79 genes
{'LINC01776', 'EEF1A1P11', 'AL589740.1', 'PAUPAR', 'MAPT', 'ESRRB', 'AC021192.2', 'LINC02790', 'AL117332.1', 'SERPINA2', 'PTBP2', 'XRN2', 'EGR2', 'RNU1-130P', 'BEND4', 'AL139300.2', 'AC095031.1', 'AC120036.1', 'RSU1', 'CADPS', 'GALNT10', 'C8orf74', 'GALNT1', 'AC013287.1', 'NRGN', 'U95743.1', 'OLFM4', 'AC090987.1', 'RN7SL831P', 'NEGR1', 'MROH5', 'AL133270.1', 'KCNN2', 'NKX2-4', 'AL391117.1', 'AC092506.1', 'RSRC1', 'RNU7-160P', 'SRPK2', 'AL591888.1', 'AC099520.1', 'ITIH4', 'ZNF568', 'AL022724.3', 'AC026320.3', 'AC005070.3', 'NKX2-2', 'ZNF877P', 'LINC02058', 'AL032822.1', 'ASAP1', 'PINX1', 'AL136456.1', 'GUCY1A2', 'AC025839.1', 'PCDH9', 'LINC00461', 'KMT2E', 'SGO1-AS1', 'KANSL1', 'ADTRP', 'AL035078.4', 'RBFOX1', 'SOX7', 'MACROD2', 'AC022387.1', 'FGF12', 'SERPINA1', 'KIZ', 'GABBR1', 'ITIH3', 'LOC102723661', 'LINC02210-CRHR1', 'AL139093.1', 'RP1L1', 'PKP4', 'U91319.1', 'NEDD4L', 'SORCS3'}


Let us read in the set of SCZ GWAS genes!

In [7]:
clozukpath = '/home/attila/projects/bsm/resources/CLOZUK/supp-table-4.csv'
scz_gwas_genes = operations.get_geneset(df=pd.read_csv(clozukpath, skiprows=7), col='Gene(s) tagged')

In [8]:
querydict = {'near_gens_Annotation': ['coding nonsyn', 'coding syn', 'missense', 'stop-gain', 'intronic (splice_site)'],
             'ensembl_Predicted Function': ['coding'],
             'sift_Prediction': ['Deleterious', 'Deleterious - Low Confidence'],
             'polyphen_Prediction': ['Probably Damaging', 'Possibly Damaging'],
             'tfbs_TFBS Name': None,
             'phast_Score': None,
             'gerp_Element RS Score': None,
             'cpg_CpG Island': None,
             'near_gens_Overlapped Gene': {'SCZ GWAS genes': scz_gwas_genes,
                                           'ASD GWAS genes': asd_gwas_genes,
                                           'ASD reported GWAS genes': reported,
                                          },
            }
results = operations.multiquery(querydict, data, do_sum=False, do_sort=False)
summary = operations.summarize_query_results(results, data, chisq=True, margin=False)
summary#.style.bar(subset='chisq stat')

/home/attila/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/home/attila/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:5982: RuntimeWarning: invalid value encountered in true_divide
  terms = (f_obs.astype(np.float64) - f_exp)**2 / f_exp


Control  SCZ  ASD  \
Feature                    Query                                             
near_gens_Annotation       coding nonsyn                      23   43   23   
                           coding syn                         16   20   15   
                           missense                            0    0    0   
                           stop-gain                           1    0    0   
                           intronic (splice_site)              0    1    0   
ensembl_Predicted Function coding                             39   64   40   
sift_Prediction            Deleterious                         8    9    6   
                           Deleterious - Low Confidence        1    1    2   
polyphen_Prediction        Probably Damaging                   3    5    2   
                           Possibly Damaging                   2    2    0   
tfbs_TFBS Name             any                                48   78   50   
phast_Score                any                               102  174  141   
gerp_Element RS Score      any                               140  262  208   
cpg_CpG Island             any                                26   47   19   
near_gens_Overlapped Gene  SCZ GWAS genes                     20   66   42   
                           ASD GWAS genes                      2   20    7   
                           ASD reported GWAS genes             0    6    0   

                                                         chisq stat   chisq p  
Feature                    Query                                               
near_gens_Annotation       coding nonsyn                   4.480340  0.106440  
                           coding syn                      1.177849  0.554924  
                           missense                             NaN       NaN  
                           stop-gain                       2.887477  0.236044  
                           intronic (splice_site)          1.582797  0.453210  
ensembl_Predicted Function coding                          3.794387  0.149989  
sift_Prediction            Deleterious                     1.315056  0.518131  
                           Deleterious - Low Confidence    0.429823  0.806613  
polyphen_Prediction        Probably Damaging               1.080625  0.582566  
                           Possibly Damaging               2.470275  0.290795  
tfbs_TFBS Name             any                             4.120011  0.127453  
phast_Score                any                             1.591145  0.451323  
gerp_Element RS Score      any                             5.012216  0.081585  
cpg_CpG Island             any                             9.614759  0.008169  
near_gens_Overlapped Gene  SCZ GWAS genes                 10.800565  0.004515  
                           ASD GWAS genes                 11.912735  0.002589  
                           ASD reported GWAS genes         9.496785  0.008666

In [9]:
%connect_info

{
  "shell_port": 41725,
  "iopub_port": 36167,
  "stdin_port": 60409,
  "control_port": 40337,
  "hb_port": 52123,
  "ip": "127.0.0.1",
  "key": "96401c45-fa6385b36e8d4ec92ba12e38",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-3022da89-61f5-46b7-b6e2-9f710614c66e.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
